In [0]:
import pandas as pd
from tqdm import notebook
notebook.tqdm.pandas()

In [0]:
R_DF = pd.read_csv('/content/drive/My Drive/Data Science Projects/RecipeApp/data/myData/ALL_RECIPES_FINAL.csv', index_col=0)

In [0]:
I_DF = pd.read_csv('/content/drive/My Drive/Data Science Projects/RecipeApp/data/myData/INGR_FINAL_COMPL_MERGE.csv', index_col=0)

In [0]:
F_DF = pd.read_csv('/content/drive/My Drive/Data Science Projects/RecipeApp/data/FoodData_Central/food.csv', index_col=0)

In [0]:
I_DF = I_DF.merge(F_DF, on='fdc_id', how='left')

In [0]:
R_DF = R_DF.drop(['ingredients', 'instructions', 'total_time'], axis=1)

In [0]:
R_I_DF = R_DF.merge(I_DF, left_on=R_DF.index, right_on='recipe_index')

## Encode ingredients and nutrition data to the df

In [0]:
# R_DF['total_grams'] = 0.0
# R_DF['protein_percent'] = 0.0
# R_DF['carb_percent'] = 0.0
# R_DF['fat_percent'] = 0.0

for i in range(1,46):
  R_DF['ingr_' + str(i)] = ''
  # R_DF['percent_' + str(i)] = 0.0
  # R_DF['protein_' + str(i)] = 0.0
  # R_DF['carbs_' + str(i)] = 0.0
  # R_DF['fat_' + str(i)] = 0.0



In [24]:
R_DF

,title,rating,yeilds,category,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,ingr_10,ingr_11,ingr_12,ingr_13,ingr_14,ingr_15,ingr_16,ingr_17,ingr_18,ingr_19,ingr_20,ingr_21,ingr_22,ingr_23,ingr_24,ingr_25,ingr_26,ingr_27,ingr_28,ingr_29,ingr_30,ingr_31,ingr_32,ingr_33,ingr_34,ingr_35,ingr_36,ingr_37,ingr_38,ingr_39,ingr_40,ingr_41,ingr_42,ingr_43,ingr_44,ingr_45
0,Bacon Bourbon Apple Pancakes,4.83,4.0,"['Home', 'Recipes', 'Breakfast and Brunch', 'P...",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Greek Yogurt Blueberry-Lemon Pancakes,5.00,NaN,"['Home', 'Recipes', 'Breakfast and Brunch', 'P...",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,Hearty Country Hot Cakes,4.34,NaN,"['Home', 'Recipes', 'Breakfast and Brunch', 'P...",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,Oatmeal-Banana Pancakes,4.67,NaN,"['Home', 'Recipes', 'Breakfast and Brunch', 'P...",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,Skillet Strawberry Pancake,4.36,NaN,"['Home', 'Recipes', 'Breakfast and Brunch', 'P...",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71021,S'mores Mega Milkshake,5.00,NaN,"['Home', 'Recipes', 'Drinks', 'Shakes and Floa...",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
71022,Cucumber Tea Spritzer,5.00,NaN,"['Home', 'Recipes', 'Drinks', 'Tea', 'Iced']",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
71023,Capatini (Cappuccino Martini),5.00,NaN,"['Home', 'Recipes', 'Drinks', 'Coffee']",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
71024,Hot Nutty Irish Coffee,5.00,NaN,"['Home', 'Recipes', 'Drinks', 'Coffee']",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [9]:
def encode_ingredients(df, I_DF):
  for index, row in notebook.tqdm(df.iterrows(), total=len(df)):
    try:
      ingredient_df = I_DF[I_DF.recipe_index == index]#.sort_values(by=['grams'], ascending=False)
      # display(ingredient_df)
      # display(ingredient_df)
      j = 0
      # total_grams = get_total('grams', ingredient_df) 
      # R_DF.at[index, 'protein_percent'] = get_total('protein', ingredient_df) / total_grams
      # R_DF.at[index, 'carb_percent'] = get_total('carbs', ingredient_df) / total_grams
      # R_DF.at[index, 'fat_percent'] = get_total('fat', ingredient_df)/ total_grams
      for index1, ingredient in ingredient_df.iterrows():
        j = j + 1
        try:
          df.at[index, 'ingr_' + str(j)] = ingredient['description']
        except:
          df.at[index, 'ingr_' + str(j)] = ''
      #   # R_DF.at[index, 'percent_' + str(j)] = ingredient['grams'] / total_grams
      #   # R_DF.at[index, 'protein_' + str(j)] = ingredient['protein']
      #   # R_DF.at[index, 'carbs_' + str(j)] = ingredient['carbs']
      #   # R_DF.at[index, 'fat_' + str(j)] = ingredient['fat'] 
    except:
      continue

  return df

R_DF = encode_ingredients(R_DF, I_DF)


In [0]:
R_DF.to_csv('/content/drive/My Drive/Data Science Projects/RecipeApp/data/myData/R_DF.csv')

In [2]:
R_DF = pd.read_csv('/content/drive/My Drive/Data Science Projects/RecipeApp/data/myData/R_DF.csv', index_col=0)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,52,53,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
R_DF = R_DF[R_DF.ingr_1.isnull() == False]

In [0]:
R_DF = R_DF.drop(columns=['title', 'rating', 'yeilds'])

In [0]:
R_DF = R_DF.fillna('')

In [0]:
df = R_DF[R_DF.category.str.contains('Soup')]


In [30]:
!pip install ctgan
def train_model(df, epochs):
  keep_cols = ['category']
  for i in range(1,20):
    keep_cols.append('ingr_' + str(i))
  df = df[keep_cols]

  discrete_columns = [
      'category'
  ]

  for i in range(1,20):
    discrete_columns.append('ingr_' + str(i))

  # df = df.dropna()

  print(discrete_columns)

  from ctgan import CTGANSynthesizer

  ctgan = CTGANSynthesizer()
  ctgan.fit(df, discrete_columns, epochs=epochs)
  return ctgan

  

In [31]:
# df = R_DF[R_DF.category.str.contains("'Home', 'Recipes', 'Breakfast and Brunch', 'Pancakes'")]
# df = R_DF.head(20000)
# df
model = train_model(df, 1000)

['category', 'ingr_1', 'ingr_2', 'ingr_3', 'ingr_4', 'ingr_5', 'ingr_6', 'ingr_7', 'ingr_8', 'ingr_9', 'ingr_10', 'ingr_11', 'ingr_12', 'ingr_13', 'ingr_14', 'ingr_15', 'ingr_16', 'ingr_17', 'ingr_18', 'ingr_19']
Epoch 1, Loss G: 5.9587, Loss D: -0.3275
Epoch 2, Loss G: 5.9095, Loss D: -0.7959
Epoch 3, Loss G: 6.0391, Loss D: -1.3786
Epoch 4, Loss G: 5.9450, Loss D: -2.4140
Epoch 5, Loss G: 5.8832, Loss D: -3.4529
Epoch 6, Loss G: 5.4584, Loss D: -4.3048
Epoch 7, Loss G: 4.6337, Loss D: -4.3317
Epoch 8, Loss G: 3.7192, Loss D: -3.7667
Epoch 9, Loss G: 3.3307, Loss D: -3.5980
Epoch 10, Loss G: 2.9874, Loss D: -2.9690
Epoch 11, Loss G: 2.2988, Loss D: -2.1504
Epoch 12, Loss G: 1.6593, Loss D: -0.9393
Epoch 13, Loss G: 1.2288, Loss D: -0.3250
Epoch 14, Loss G: 1.5989, Loss D: 0.6704
Epoch 15, Loss G: 2.2432, Loss D: 0.6189
Epoch 16, Loss G: 3.0359, Loss D: 0.5683
Epoch 17, Loss G: 4.0716, Loss D: 0.2616
Epoch 18, Loss G: 4.7751, Loss D: 0.0477
Epoch 19, Loss G: 5.5968, Loss D: -0.0783
Epo

In [0]:
sample = model.sample(50)

In [33]:
sample

,category,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,ingr_10,ingr_11,ingr_12,ingr_13,ingr_14,ingr_15,ingr_16,ingr_17,ingr_18,ingr_19
0,"['Home', 'Recipes', 'Soups, Stews and Chili', ...","Butter, NFS",RICE FLOUR,"Onions, mature, raw",SCALLOPS,"Oil, sesame, salad or cooking","Spices, cinnamon, ground","Wheat flour, white, all-purpose, unenriched","Water, tap",PUFF PASTRY SHEETS,"Applesauce, canned, sweetened, with salt","Onions, sweet, raw","Salt, table",,,,,,,
1,"['Home', 'Recipes', 'Soups, Stews and Chili', ...","Onions, raw","Cereals, QUAKER, Quick Oats, Dry",,Olive oil,"Cucumber, with peel, raw","Garlic, raw","Milk, whole","Egg, quail, whole, fresh, raw",,,"Butter, NFS",,,,,,,,
2,"['Home', 'Recipes', 'Soups, Stews and Chili', ...",SLICED BACON,"Onions, raw","Spices, pepper, black","Squash, winter, butternut, raw","Mushrooms, white, raw","Spices, oregano, dried","Milk, whole, 3.25% milkfat, with added vitamin D","Peppers, sweet, red, raw","Oil, sesame, salad or cooking","Water, tap","Spices, cinnamon, ground","Spices, pepper, black","Salt, table",CRUSHED RED PEPPER,"Lemons, raw, without peel",,,,
3,"['Home', 'Recipes', 'Soups, Stews and Chili', ...","Water, tap","Bananas, raw",,"Spices, cloves, ground","Peppers, sweet, green, raw","Salt, table","Carrots, raw","Coriander (cilantro) leaves, raw","Garlic, raw",,"Spices, thyme, dried","Spices, paprika",,,,"Tomato products, canned, paste, without salt a...",,,
4,"['Home', 'Recipes', 'Soups, Stews and Chili', ...",ALL-PURPOSE BAKING FLOUR,ALMONDS,"Milk, whole, 3.25% milkfat, with added vitamin D","Nuts, almonds","Shallots, raw",,"Leavening agents, baking powder, low-sodium",,TACO SEASONING,,,"Salt, table","Romaine lettuce, raw",,,,,,
5,"['Home', 'Recipes', 'Soups, Stews and Chili', ...",RICE WINE VINEGAR,"Coriander (cilantro) leaves, raw","Cauliflower, raw",Pork sausage,"Spices, basil, dried",PEPPERS,"Salad dressing, italian dressing, commercial, ...",SEAFOOD SEASONING,"Butter, stick, unsalted",,,"Beverages, coconut milk, sweetened, fortified ...",,,,,,,
6,"['Home', 'Recipes', 'Soups, Stews and Chili', ...","Onions, raw","Sugars, granulated","Onions, raw","Soup, chicken broth, ready-to-serve","Spices, basil, dried",RICE VINEGAR,"Spices, basil, dried",,,,,,,,,"Lemon juice, raw",,"Sugars, brown",ADOBO SEASONING
7,"['Home', 'Recipes', 'Soups, Stews and Chili', ...",Soy milk,"Pork, cured, bacon, unprepared",APPLE,Olive oil,"Horseradish, prepared","Butter, salted","Spices, pepper, red or cayenne","Salt, table",GREEN BEANS,"Cream, sour, cultured","Spices, chili powder",,,,,"Leavening agents, baking soda",,,
8,"['Home', 'Recipes', 'Soups, Stews and Chili', ...","Mayonnaise, regular","Onions, raw","Celery, raw",Olive oil,,GRATED PARMESAN CHEESE,,,,,,,,,,,,,
9,"['Home', 'Recipes', 'Soups, Stews and Chili', ...","Crustaceans, shrimp, raw","Spices, cloves, ground","Butter, stick, unsalted","Sugar, white, granulated or lump","Salt, table",GREAT NORTHERN BEANS,,"Salt, table",,,,,,,,,,,


In [0]:
import requests 

def read_synthetic_data(df):
  j = 0
  for index, row in df.iterrows():
    j = j + 1
    print(j / len(df) * 100, '%')
    URL = "https://api.nal.usda.gov/fdc/v1/search?api_key=yuaqggVFg636ArnOHtePTc32tEhGOLyuSTkP56MH"
    for i in range(1, 20):
      if row['ingr_' + str(i)] == '':
        continue
      PARAMS = {"generalSearchInput":row['fdc_id_' + str(i)]} 
      data = requests.get(url = URL, params = PARAMS).json()
      print(data)
      if len(data['foods']) != 0:
        df.at[index, 'ingr_'+ str(i)] = data['foods'][0]['description']
      else:
        df.at[index, 'ingr_'+ str(i)] = ''

  return df

df = read_synthetic_data(model.sample(20))

5.0 %


KeyError: ignored

In [0]:
import pickle

with open('/content/drive/My Drive/Data Science Projects/RecipeApp/Models/model_pickle_v4.1', 'wb') as f:
  pickle.dump(ctgan, f)

In [0]:
with open('/content/drive/My Drive/Data Science Projects/RecipeApp/Models/model_pickle', 'rb') as f:
  mp = pickle.load(f)

In [0]:
pd.set_option('display.max_colwidth', 100)
df

,fdc_id_1,fdc_id_2,fdc_id_3,fdc_id_4,fdc_id_5,fdc_id_6,fdc_id_7,fdc_id_8,fdc_id_9,fdc_id_10,fdc_id_11,fdc_id_12,fdc_id_13,fdc_id_14,fdc_id_15,fdc_id_16,fdc_id_17,fdc_id_18,fdc_id_19,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,ingr_10,ingr_11,ingr_12,ingr_13,ingr_14,ingr_15,ingr_16,ingr_17,ingr_19,ingr_18
0,167762,169761,170108,170393,339003,169761,339003,170000,343812,343812,173647,170416,173468,173468,169656,168833,173468,999999,171328,"Strawberries, raw","Wheat flour, white, all-purpose, unenriched","Peppers, sweet, red, raw","Carrots, raw","Egg, whole, raw","Wheat flour, white, all-purpose, unenriched","Egg, whole, raw","Onions, raw","Butter, NFS","Butter, NFS","Beverages, water, tap, drinking","Parsley, fresh","Salt, table","Salt, table","Sugars, powdered","Sugars, brown","Salt, table","Spices, oregano, dried",NaN
1,175179,172183,168881,564782,343816,168166,170859,489193,342614,173469,170000,386272,173468,343930,172805,999999,171329,999999,999999,"Crustaceans, shrimp, raw","Egg, white, raw, fresh","Rice, white, short-grain, enriched, uncooked",CHICKEN BREASTS,"Butter, stick, unsalted","Raisins, seeded","Cream, fluid, heavy whipping",SLICED GREEN OLIVES,"Garlic, raw","Vinegar, cider","Onions, raw",CHILI GARLIC SAUCE,"Salt, table","Sugar, white, granulated or lump","Leavening agents, baking powder, low-sodium",NaN,"Spices, paprika",NaN,NaN
2,173418,169761,343812,173468,339003,170459,169251,343930,999999,344526,343873,344526,999999,343930,170926,173468,999999,999999,999999,"Cheese, cream","Wheat flour, white, all-purpose, unenriched","Butter, NFS","Salt, table","Egg, whole, raw","Tomato products, canned, paste, without salt added (Includes foods for USDA's Food Distribution ...","Mushrooms, white, raw","Sugar, white, granulated or lump",NaN,"Water, tap",Olive oil,"Water, tap",NaN,"Sugar, white, granulated or lump","Spices, ginger, ground","Salt, table",NaN,NaN,NaN
3,344526,344526,344526,344526,339003,343672,458232,169593,175040,343873,418653,173468,343930,999999,999999,512689,999999,999999,999999,"Water, tap","Water, tap","Water, tap","Water, tap","Egg, whole, raw",Hot pepper sauce,INSTANT MASHED POTATO FLAKES,"Cocoa, dry powder, unsweetened","Leavening agents, baking soda",Olive oil,BLANCHED SLIVERED ALMONDS,"Salt, table","Sugar, white, granulated or lump",NaN,NaN,CUMIN GROUND,NaN,NaN,NaN
4,342493,171155,171790,170026,175179,170427,170931,170002,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,"Sweet potato, canned","Soup, cream of mushroom, canned, condensed","Beef, ground, 95% lean meat / 5% fat, raw","Potatoes, flesh and skin, raw","Crustaceans, shrimp, raw","Peppers, sweet, green, raw","Spices, pepper, black","Onions, dehydrated flakes",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,169941,173647,344526,173418,478456,168820,343866,170393,343812,172237,343812,344526,175040,173468,171326,173468,173468,169997,999999,"Persimmons, japanese, raw","Beverages, water, tap, drinking","Water, tap","Cheese, cream",SAUSAGES,Molasses,"Vegetable oil, NFS","Carrots, raw","Butter, NFS","Vinegar, distilled","Butter, NFS","Water, tap","Leavening agents, baking soda","Salt, table","Spices, nutmeg, ground","Salt, table","Salt, table",NaN,"Coriander (cilantro) leaves, raw"
6,173647,175179,173741,170900,453250,343866,384528,169988,170006,342614,344522,999999,999999,999999,999999,999999,999999,999999,999999,"Beverages, water, tap, drinking","Crustaceans, shrimp, raw","Beans, kidney, all types, mature seeds, canned","Cheese, mozzarella, low moisture, part-skim, shredded",ITALIAN SEASONED BREAD CRUMBS,"Vegetable oil, NFS",CORN,"Celery, raw","Onions, young green, tops only","Garlic, raw",Rum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,170427,344526,567566,343930,174523,170000,999999,171610,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,999999,"Peppers, sweet, green, raw","Water, tap",CHEDDAR CHEESE,"Sugar, white, granulated or lump","Sauce, barbecue","Onions, raw",NaN,"Sauce, worcestershir